## Python for Stock Options 1 | Advanced Options Strategies Python 

In [1]:
### Installing libraries
# !pip install yahoofinancials mibian selenium webdriver-manager 

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import mibian
from yahoofinancials import YahooFinancials
import matplotlib.pyplot as plt

In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

In [20]:
today = datetime.today() - timedelta(days=1)
today

datetime.datetime(2024, 7, 11, 11, 55, 51, 847386)

In [5]:
# yahoo_financials_treasuries = YahooFinancials('^TNX')
risk_free_rate = 0.0419 #round(yahoo_financials_treasuries.get_current_price()/100, 4)
risk_free_rate

0.0419

In [6]:
stock = "AAPL"

In [7]:
url = f"https://finance.yahoo.com/quote/{stock}/options/?straddle=false"

In [10]:
def get_target_date(experations):
    today = datetime.today() - timedelta(days=1)
    experations_dates = [datetime.strptime(i.text, "%b %d, %Y") for i in experations if len(i.text)>5 and i.text[-4:]=='2024']
    start_date = today + timedelta(days=60)
    end_date = today + timedelta(days=80)
    filtered_dates = [date for date in experations_dates if start_date <= date <= end_date]
    return filtered_dates[0]

In [11]:
driver.get(url)
# driver.implicitly_wait(5)
experations_btn = driver.find_element(By.XPATH, '/html/body/div[1]/main/section/section/section/article/section[2]/div/div[1]/button').click()
experations = driver.find_elements(By.CLASS_NAME, 'itm.svelte-5qjwyh')
target_date_dt = get_target_date(experations)

# Desired date as a string
target_date_str = target_date_dt.strftime('%b %d, %Y')

# Loop through date elements to find the desired date and click on it
for element in experations:
    if element.text == target_date_str:
        element.click()
        break
driver.implicitly_wait(5)
tables = driver.find_elements(By.TAG_NAME, "table")
stock_price = driver.find_element(By.CLASS_NAME, "livePrice").text

In [12]:
calls = [i.text.split() for i in tables[0].find_elements(By.TAG_NAME, 'tr')]
puts = [i.text.split() for i in tables[1].find_elements(By.TAG_NAME, 'tr')]

In [13]:
column_names = ['Contract', 'Date', 'Time', 'ET', 'Strike', 'Price', 'Bid', 'Ask', 'Change', '% Change', 'Volume', 'Open Interest', 'Implied Volatility']
calls_df = pd.DataFrame(calls[1:], columns=column_names)
puts_df = pd.DataFrame(puts[1:], columns=column_names)

In [14]:
calls_df['Strike'] = calls_df['Strike'].astype(float)
puts_df['Strike'] = puts_df['Strike'].astype(float)

In [15]:
stock_price = float(stock_price)

In [16]:
lower = calls_df[calls_df['Strike'] < stock_price].tail(3)[::-1]
higher = calls_df[calls_df['Strike'] > stock_price].head(3)[::-1]

In [17]:
calls_df = pd.concat([higher, lower])
calls_df.reset_index(drop=True, inplace=True)

In [18]:
calls_df

,Contract,Date,Time,ET,Strike,Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL240920C00240000,7/11/2024,7:59,PM,240.0,5.30,0.00,0.00,0.00,0.00%,"6,462",0,3.13%
1,AAPL240920C00235000,7/11/2024,7:59,PM,235.0,7.15,0.00,0.00,0.00,0.00%,"2,932",0,1.56%
2,AAPL240920C00230000,7/11/2024,7:59,PM,230.0,9.33,0.00,0.00,0.00,0.00%,"9,352",0,0.78%
3,AAPL240920C00225000,7/11/2024,7:59,PM,225.0,12.02,0.00,0.00,0.00,0.00%,"1,450",0,0.00%
4,AAPL240920C00220000,7/11/2024,7:58,PM,220.0,15.17,0.00,0.00,0.00,0.00%,"11,239",0,0.00%
5,AAPL240920C00215000,7/11/2024,7:59,PM,215.0,18.75,0.00,0.00,0.00,0.00%,993,0,0.00%


In [21]:
maturity = (target_date_dt - today).days

In [22]:
calls_df['IV'] = calls_df['Implied Volatility'].str.strip("%").astype(float)

In [23]:
def calculate_option_value(row):
    strike = row['Strike']
    volatility = row['IV']

    option = mibian.BS([stock_price, strike, risk_free_rate, maturity], volatility=volatility)
    return [
        option.callPrice,
        option.callDelta,
        option.callTheta,
        option.vega,
        option.gamma,
        option.callRho
    ]

In [24]:
calls_df[['BSM Price', 'Call Delta', 'Call Theta', 'Vega', 'Gamma', 'Rho']] = calls_df[['Strike', 'IV']].apply(calculate_option_value, axis=1, result_type='expand')

In [25]:
calls_df.drop(columns=['Date', 'Time', 'ET', 'Change', '% Change', 'Implied Volatility'], inplace=True)

In [26]:
calls_df.columns

Index(['Contract', 'Strike', 'Price', 'Bid', 'Ask', 'Volume', 'Open Interest',
       'IV', 'BSM Price', 'Call Delta', 'Call Theta', 'Vega', 'Gamma', 'Rho'],
      dtype='object')

In [27]:
calls_df['Ask'] = calls_df['Ask'].astype(float)

In [28]:
calls_df['Under'] = np.where(calls_df['Ask']<calls_df['BSM Price'], True, False)

In [29]:
calls_df

,Contract,Strike,Price,Bid,Ask,Volume,Open Interest,IV,BSM Price,Call Delta,Call Theta,Vega,Gamma,Rho,Under
0,AAPL240920C00240000,240.0,5.30,0.00,0.0,"6,462",0,3.13,3.972850e-05,0.000055,-5.044041e-06,0.000225,0.000072,2.395880e-05,True
1,AAPL240920C00235000,235.0,7.15,0.00,0.0,"2,932",0,1.56,4.234169e-07,0.000001,-7.534739e-08,0.000007,0.000004,6.024973e-07,True
2,AAPL240920C00230000,230.0,9.33,0.00,0.0,"9,352",0,0.78,2.197288e-04,0.001020,-1.931144e-05,0.003418,0.004413,4.449138e-04,True
3,AAPL240920C00225000,225.0,12.02,0.00,0.0,"1,450",0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,False
4,AAPL240920C00220000,220.0,15.17,0.00,0.0,"11,239",0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,False
5,AAPL240920C00215000,215.0,18.75,0.00,0.0,993,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,False
